In [1]:
import os, matplotlib.pyplot as plt, joblib, copy, pandas as pd
os.environ["OMP_NUM_THREADS"] = "1"
#useful on linux to avoid numpy multithreading
 
import numpy as np, scipy, sklearn, torch, xarray as xr
torch.set_default_dtype(torch.float64)
# can be omitted or changed to the preferred Tensors precision

# imports of the repository files, ensure that they work
try: 
    import srnn, srnn_utils
    # if the package was installed via pip command  
except ImportError:
    import sys, os
    sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), "src"))
    import srnn, srnn_utils
    # if the package is not installed into the environment, make sure the "path" variable below includes "src" folder containing srnn python files. Here -- "../src"

np.random.seed(1)
torch.manual_seed(2)
# fixing random generators for stable debugging

### **1. Load the data**

In [2]:
# Path to the prepared data folder
processed_data_path=os.path.expanduser('~/data/srnn_data/MPI-ESM')
#CP = joblib.load(os.path.join(processed_data_path,'CP.jpkl'))
#pcs,kpcs,init_length = joblib.load(os.path.join(processed_data_path,'training_data.jpkl'))
evaluation_pcs, evaluation_kpcs,init_length = joblib.load(os.path.join(processed_data_path,'evaluation_data.jpkl'))

### **2. Load optimal SRNN parameters and create model objects**

In [3]:
def create_models(filename):
    parameters=joblib.load(filename)
    models=[]
    for p in parameters:
        m=srnn.StochasticRNN(dim_in=p['dim_in'],dim_out=p['dim_out'],m=p['m'])
        m.load_state_dict(p['weights'])
        models.append(m)
    lags=np.unique([p['lag'] for p in parameters])
    if len(lags)==1: 
        lag=lags[0]
    else:
        print('Lags are not equal') # Here we don't assume such a case   
    return models, lag

models, lag = create_models(os.path.join(processed_data_path,'selected_srnn_parameters_list_3a.jpkl'))
print(models[0].m,lag)

12 11


### **3. Make predictions**

There are 'nfolds' models after cross-vaa/lidation. We make a prediction function which randomly picks one of them at each elementary prediction. Thus we include the related uncertainty.

In [4]:
def predict_time_series(pcs: torch.Tensor, kpcs: torch.Tensor,
                        lag: int, models: list[srnn.StochasticRNN], n_steps: int):
    '''
    The function to predict the time series ensemble (data), starting from its last time point

    Arguments: 
    pcs[N,NT,D1], kpcs[N,NT,D2]  -- normalized time series of the duration nt; here kpcs values does not matter (they are not used)
    lag -- int, the lag to use (in this case must be the same used for model training)
    models -- list of StochasticRNN class objects
    n_steps>=0 -- int, number of prediction steps (lead time)
    
    Returns:
    
    pcs_prediction[N,n_steps+1,D1], kpcs_prediction[N,n_steps+1,D1] -- the last value of data + n_steps predicted values
    '''
    n_models=len(models)
    with torch.no_grad():
        N,NT,D1=pcs.size()
        for i in range(n_steps):
            inputs=pcs[:,-lag:,:]
            
            # We need to make prediction by all models and randomly draw from the model for each sample
            data_next_variants=[model.predict(inputs) for model in models]
            variant_number=np.random.randint(n_models,size=N)
            data_next=data_next_variants[0]
            for i in range(1,n_models): 
                data_next[variant_number==i,:,:]=data_next_variants[i][variant_number==i,:,:]

            pcs=torch.cat([pcs,data_next[:,:,:D1]],dim=-2)
            kpcs=torch.cat([kpcs,data_next[:,:,D1:]],dim=-2)
        return pcs[...,NT-1:NT+n_steps,:].numpy(), kpcs[...,NT-1:NT+n_steps,:].numpy()


Here we make long-term forecast (to forget initial conditions and produce invariant distribution for SRNN emulator).

In [5]:
nstart=init_length   # starting point in time
N=evaluation_pcs.shape[0]        # number of random realizations
n_steps=evaluation_pcs.shape[1]-init_length # lead time
n_skip=200

# Preprocess
pcs=torch.tensor(evaluation_pcs[...,:3],dtype=torch.get_default_dtype()) 
kpcs=torch.tensor(evaluation_kpcs[...,:3],dtype=torch.get_default_dtype()) 

#make prediction
predicted_pcs, predicted_kpcs = predict_time_series(pcs[:,:nstart,:],kpcs[:,:nstart,:],lag,models,n_steps=n_steps+n_skip)
predicted_pcs=predicted_pcs[:,n_skip+1:,:]
predicted_kpcs=predicted_kpcs[:,n_skip+1:,:]
files=joblib.dump((predicted_pcs,predicted_kpcs),os.path.join(processed_data_path,'predicted_data_4a.jpkl'))

Here we make short-term (trajectory) forecast to assess SRNN prediction skill.

In [6]:
n_steps=50
nstart_values=list(range(init_length,evaluation_pcs.shape[1]-n_steps))
N=100
n_years, n_days, n_pcs=pcs.shape
n_years, n_days, n_kpcs=kpcs.shape
all_predicted_pcs=[]
all_predicted_kpcs=[]
all_observed_pcs=[]
all_observed_kpcs=[]
for nstart in nstart_values:
    predicted_pcs, predicted_kpcs = predict_time_series(pcs[:,:nstart,:].expand(N,-1,-1,-1).reshape((N*n_years,nstart,n_pcs)),
                                                        kpcs[:,:nstart,:].expand(N,-1,-1,-1).reshape((N*n_years,nstart,n_kpcs)),
                                                        lag,models,n_steps=n_steps)
    all_predicted_pcs.append(predicted_pcs.reshape((N,n_years,-1,n_pcs)).mean(axis=0))
    all_predicted_kpcs.append(predicted_kpcs.reshape((N,n_years,-1,n_kpcs)).mean(axis=0))
    all_observed_pcs.append(pcs[:,nstart-1:nstart+n_steps,:].numpy())    
    all_observed_kpcs.append(kpcs[:,nstart-1:nstart+n_steps,:].numpy())

files=joblib.dump((np.array(all_predicted_pcs),
            np.array(all_predicted_kpcs),
            np.array(all_observed_pcs),
            np.array(all_observed_kpcs)),os.path.join(processed_data_path,'predicted_data_short_4a.jpkl'))